# Interaction Model Examples

This walks through the various interactions that can be initiated from the REPL towards a target using the Matter Interaction Model (IM) and Data Model (DM).

[![Launch REPL Live](https://mybinder.org/static/images/badge_logo.svg?v=51b20aa16836ea83f5ed69194c660eb85c4c2c1e32565312baedb7d534e3ffcf592881dbbe3da441d8293ded842755c906b91fb4aadf15220cf48111ebf701c4)](http://35.236.121.59/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fmrjerryjohns%2Frepl-docker&urlpath=lab%2Ftree%2Frepl-docker%2FMatter+-+Basic+Interactions.ipynb&branch=master)


## Imports

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

In [1]:
from rich import print
from rich.pretty import pprint
from rich import inspect
import logging
import chip
import chip.logging
from chip import ChipDeviceCtrl
import chip.clusters as Clusters
from chip.ChipReplStartup import *
from chip.ChipStack import *
import subprocess, sys
ReplInit()
chipStack = ChipStack()

──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

2022-01-06 14:21:10 johnsj-macbookpro1.roam.corp.google.com root[47897] CRITICAL Loading configuration from /tmp/repl-storage.json...


## Cluster Elements

The Interaction Model uses data model types that refer not just to the various base types defines in the spec, but types that correspond to structs/commands/events/attributes defined in each cluster specification. The cluster-specific types are referred to as 'cluster objects'. These are represented as Python dataclasses, with each field in the respective object equivalently named as a member within the dataclass.

### Namespaces

Objects in clusters are organized into namespaces. All clusters can be found under the `Clusters` namespace, with the appropriate cluster in upper camel case within that. (e.g `Clusters.TestCluster`).

Within that, `Commands`, `Structs` and `Attributes` delimit the respective types in the cluster.

_Example Struct:_

In [11]:
v = Clusters.TestCluster.Structs.SimpleStruct()
v.a = 20
v.b = True
v.c = Clusters.TestCluster.Enums.SimpleEnum.kValueA
v.d = b'1234'
v.e = 30
v.g = 23.234

v

SimpleStruct(
│   a=20,
│   b=True,
│   c=<SimpleEnum.kValueA: 1>,
│   d=b'1234',
│   e=30,
│   f=0,
│   g=23.234,
│   h=0.0
)

_Example Command:_

In [12]:
Clusters.TestCluster.Commands.TestAddArguments()

TestAddArguments(
│   arg1=0,
│   arg2=0
)

To get more information about the fields in these objects and their types, run:

In [13]:
matterhelp(Clusters.TestCluster.Commands.TestAddArguments)

╭──────── <class 'chip.clusters.Objects.TestCluster.Commands.TestAddArguments'> ─────────╮
│ def TestCluster.Commands.TestAddArguments(arg1: 'uint' = 0, arg2: 'uint' = 0) -> None: │
│                                                                                        │
│ TestAddArguments(arg1: 'uint' = 0, arg2: 'uint' = 0)                                   │
│                                                                                        │
│       arg1 = 0                                                                         │
│       arg2 = 0                                                                         │
│ cluster_id = 1295                                                                      │
│ command_id = 4                                                                         │
│ descriptor = ClusterObjectDescriptor(                                                  │
│                  Fields=[                                                              │
│                      ClusterObjectFieldDescriptor(                                     │
│                          Label='arg1',                                                 │
│                          Tag=0,                                                        │
│                          Type=<class 'chip.tlv.uint'>                                  │
│                      ),                                                                │
│                      ClusterObjectFieldDescriptor(                                     │
│                          Label='arg2',                                                 │
│                          Tag=1,                                                        │
│                          Type=<class 'chip.tlv.uint'>                                  │
│                      )                                                                 │
│                  ]                                                                     │
│              )                                                                         │
│  is_client = True                                                                      │
│   FromDict = def FromDict(data: dict):                                                 │
│    FromTLV = def FromTLV(data: bytes):                                                 │
│      ToTLV = def ToTLV(self):                                                          │
╰────────────────────────────────────────────────────────────────────────────────────────╯

### Nullable Fields

For fields that are nullable, they are represented as a `Typing.Union[Nullable, ...]`. This means that it can either be a `Nullable` type or the underlying type of the field.

When nullable, a field can either take on the value of the native type, or a value of `NullValue`.

In [14]:
a = Clusters.TestCluster.Structs.NullablesAndOptionalsStruct()
a.nullableInt = Clusters.Types.NullValue
a

NullablesAndOptionalsStruct(
│   nullableInt=Null,
│   optionalInt=None,
│   nullableOptionalInt=None,
│   nullableString=Null,
│   optionalString=None,
│   nullableOptionalString=None,
│   nullableStruct=Null,
│   optionalStruct=None,
│   nullableOptionalStruct=None,
│   nullableList=Null,
│   optionalList=None,
│   nullableOptionalList=None
)

### Optional Fields

If a field is optional, it is represented in the typing hints as a `Typing.Union[NoneType, ...]`. An optional field that isn't present has a value of `None`.

In [15]:
print(a.nullableOptionalInt)

None

## IM Interactions

This section will walk through the various types of IM Interactions that are possible in the REPL.

### Commission and Setup Server

#### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [2]:
import time
subprocess.Popen(['pkill', '-f', 'chip-all-clusters-app'])
process = subprocess.Popen('./out/debug/chip-all-clusters-app', stdout=subprocess.DEVNULL)
time.sleep(1)

#### Commission Target

Commission the target with a NodeId of 1.

In [3]:
devCtrl = ChipDeviceCtrl.ChipDeviceController(controllerNodeId=1)
devCtrl.CommissionIP(b'127.0.0.1', 20202021, 1)

2022-01-06 14:21:22 johnsj-macbookpro1.roam.corp.google.com chip.CTL[47897] ERROR Unable to find country code, defaulting to WW


Established CASE with Device
Node address has been updated
Commissioning complete


True

### Invoke Interaction

#### Basic Command (Success Response)

Let's send a basic command to turn on/off the light on Endpoint 1.

In [18]:
await devCtrl.SendCommand(1, 1, Clusters.OnOff.Commands.On())

The receipt of a successful status response will result in the command just returning successfully. Otherwise, an exception will be thrown.

#### Basic Command (Failure Response)

If we send the same command to an invalid endpoint, an exception is thrown. If an IM status code was received from the server, a `InteractionModelError` is thrown containing the IM status code:

In [19]:
await devCtrl.SendCommand(1, 100, Clusters.OnOff.Commands.On())

InteractionModelError: InteractionModelError: Failure (0x1)

#### Basic Command (Data Response)

Here's an example of a command that sends back a data response, and how that is presented:

In [20]:
await devCtrl.SendCommand(1, 1, Clusters.TestCluster.Commands.TestListInt8UReverseRequest([1, 3, 5, 7]))

TestListInt8UReverseResponse(
│   arg1=[
│   │   7,
│   │   5,
│   │   3,
│   │   1
│   ]
)

### Read Interaction

The `ReadAttribute` method on the `DeviceController` class can be used to read attributes from a target. The NodeId of the target is the first argument, followed by a list of paths that are expressed as cluster object namespaces to the respective slices of the data that is requested.

By default, the data is returned as a dictionary, with the top-level item representing the endpoint, then the cluster and the attribute. The latter two keys are expressed using cluster object namespaces.

#### Read 1 attribute:

In [37]:
a = await devCtrl.ReadAttribute(1, [Clusters.TestCluster.Attributes.Int16u])

In [28]:
a[1][Clusters.TestCluster]

{
│   <class 'chip.clusters.Objects.TestCluster.Attributes.Int16u'>: 0
}

In [29]:
a[1][Clusters.TestCluster][Clusters.TestCluster.Attributes.Int16u]

0

#### Read 2 attributes:

In [24]:
await devCtrl.ReadAttribute(1, [Clusters.TestCluster.Attributes.Int16u, Clusters.TestCluster.Attributes.Boolean])

{
│   1: {
│   │   <class 'chip.clusters.Objects.TestCluster'>: {
│   │   │   <class 'chip.clusters.Objects.TestCluster.Attributes.Boolean'>: False,
│   │   │   <class 'chip.clusters.Objects.TestCluster.Attributes.Int16u'>: 0
│   │   }
│   }
}

#### Read the entirety of a cluster on an endpoint:

The path is represented as tuple of (endpoint, cluster)

In [36]:
await devCtrl.ReadAttribute(1, [(1, Clusters.OnOff)])

{
│   1: {
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   }
│   }
}

#### Read the entirety of a cluster across all endpoints:

In [16]:
await devCtrl.ReadAttribute(1, [Clusters.OnOff])

{
│   1: {
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: False,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   }
│   },
│   2: {
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: False,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   }
│   }
}

#### Read an endpoint:

In [17]:
await devCtrl.ReadAttribute(1, [2])

{
│   2: {
│   │   <class 'chip.clusters.Objects.Groups'>: {
│   │   │   <class 'chip.clusters.Objects.Groups.Attributes.NameSupport'>: 128,
│   │   │   <class 'chip.clusters.Objects.Groups.Attributes.ClusterRevision'>: 3
│   │   },
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: False,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   },
│   │   <class 'chip.clusters.Objects.Descriptor'>: {
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.DeviceList'>: [
│   │   │   │   DeviceType(
│   │   │   │   │   type=22,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ServerList'>: [
│   │   │   │   4,
│   │   │   │   6,
│   │   │   │   29,
│   │   │   │   1030
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ClientList'>: [],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.PartsList'>: [],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ClusterRevision'>: 1
│   │   },
│   │   <class 'chip.clusters.Objects.OccupancySensing'>: {
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.Occupancy'>: 0,
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.OccupancySensorType'>: 0,
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.OccupancySensorTypeBitmap'>: 1,
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.ClusterRevision'>: 2
│   │   }
│   }
}

#### Read the entire node:

In [42]:
await devCtrl.ReadAttribute(1, [('*')])

{
│   0: {
│   │   <class 'chip.clusters.Objects.Identify'>: {
│   │   │   <class 'chip.clusters.Objects.Identify.Attributes.IdentifyTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.Identify.Attributes.IdentifyType'>: 2,
│   │   │   <class 'chip.clusters.Objects.Identify.Attributes.ClusterRevision'>: 2
│   │   },
│   │   <class 'chip.clusters.Objects.Groups'>: {
│   │   │   <class 'chip.clusters.Objects.Groups.Attributes.NameSupport'>: 128,
│   │   │   <class 'chip.clusters.Objects.Groups.Attributes.ClusterRevision'>: 3
│   │   },
│   │   <class 'chip.clusters.Objects.Descriptor'>: {
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.DeviceList'>: [
│   │   │   │   DeviceType(
│   │   │   │   │   type=22,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ServerList'>: [
│   │   │   │   3,
│   │   │   │   4,
│   │   │   │   29,
│   │   │   │   30,
│   │   │   │   31,
│   │   │   │   40,
│   │   │   │   41,
│   │   │   │   42,
│   │   │   │   46,
│   │   │   │   48,
│   │   │   │   49,
│   │   │   │   50,
│   │   │   │   51,
│   │   │   │   52,
│   │   │   │   53,
│   │   │   │   54,
│   │   │   │   55,
│   │   │   │   60,
│   │   │   │   62,
│   │   │   │   63,
│   │   │   │   64,
│   │   │   │   65,
│   │   │   │   1029
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ClientList'>: [],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.PartsList'>: [
│   │   │   │   1,
│   │   │   │   2
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ClusterRevision'>: 1
│   │   },
│   │   <class 'chip.clusters.Objects.Binding'>: {
│   │   │   <class 'chip.clusters.Objects.Binding.Attributes.ClusterRevision'>: 1
│   │   },
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [],
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Extension'>: [],
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.ClusterRevision'>: 1
│   │   },
│   │   <class 'chip.clusters.Objects.Basic'>: {
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.InteractionModelVersion'>: 1,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.VendorName'>: 'TEST_VENDOR',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.VendorID'>: 9050,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ProductName'>: 'TEST_PRODUCT',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ProductID'>: 65279,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.NodeLabel'>: '',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.Location'>: 'XX',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.HardwareVersion'>: 0,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.HardwareVersionString'>: 'TEST_VERSION',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.SoftwareVersion'>: 0,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.SoftwareVersionString'>: 'prerelease',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ManufacturingDate'>: '20210614123456ZZ',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.PartNumber'>: '',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ProductURL'>: '',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ProductLabel'>: '',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.SerialNumber'>: 'TEST_SN',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.LocalConfigDisabled'>: False,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.Reachable'>: True,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.UniqueID'>: '',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ClusterRevision'>: 3
│   │   },
│   │   <class 'chip.clusters.Objects.OtaSoftwareUpdateProvider'>: {
│   │   │   <class 'chip.clusters.Objects.OtaSoftwareUpdateProvider.Attri

#### Alternative 'Cluster' View

The above encapsulates each attribute as a 'cluster-object' key within the top-level cluster instance. Instead, an alternative view each attribute is represented as a field in the object can be retrieved by passing in `True` to the third argument:

In [43]:
await devCtrl.ReadAttribute(1, [2], True)

{
│   2: {
│   │   <class 'chip.clusters.Objects.Groups'>: Groups(
│   │   │   nameSupport=128,
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=3
│   │   ),
│   │   <class 'chip.clusters.Objects.OnOff'>: OnOff(
│   │   │   onOff=False,
│   │   │   globalSceneControl=True,
│   │   │   onTime=0,
│   │   │   offWaitTime=0,
│   │   │   startUpOnOff=0,
│   │   │   attributeList=None,
│   │   │   featureMap=0,
│   │   │   clusterRevision=4
│   │   ),
│   │   <class 'chip.clusters.Objects.Descriptor'>: Descriptor(
│   │   │   deviceList=[
│   │   │   │   DeviceType(
│   │   │   │   │   type=22,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   serverList=[
│   │   │   │   4,
│   │   │   │   6,
│   │   │   │   29,
│   │   │   │   1030
│   │   │   ],
│   │   │   clientList=[],
│   │   │   partsList=[],
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=1
│   │   ),
│   │   <class 'chip.clusters.Objects.OccupancySensing'>: OccupancySensing(
│   │   │   occupancy=0,
│   │   │   occupancySensorType=0,
│   │   │   occupancySensorTypeBitmap=1,
│   │   │   pirOccupiedToUnoccupiedDelay=None,
│   │   │   pirUnoccupiedToOccupiedDelay=None,
│   │   │   pirUnoccupiedToOccupiedThreshold=None,
│   │   │   ultrasonicOccupiedToUnoccupiedDelay=None,
│   │   │   ultrasonicUnoccupiedToOccupiedDelay=None,
│   │   │   ultrasonicUnoccupiedToOccupiedThreshold=None,
│   │   │   physicalContactOccupiedToUnoccupiedDelay=None,
│   │   │   physicalContactUnoccupiedToOccupiedDelay=None,
│   │   │   physicalContactUnoccupiedToOccupiedThreshold=None,
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=2
│   │   )
│   }
}

#### Read Events:

A `ReadEvents` API exists that behaves similarly to the `ReadAttributes` API. It permits the same degrees of wildcard expression as its counterpart and follows the same format for expressing all wildcard permutations.

#### Read all events:

In [6]:
# Force an event to get emitted.
await devCtrl.SendCommand(1, 1, Clusters.TestCluster.Commands.TestEmitTestEventRequest())

await devCtrl.ReadEvent(1, [('*')])

[
│   EventReadResult(
│   │   Header=EventHeader(
│   │   │   EndpointId=1,
│   │   │   Event=None,
│   │   │   EventNumber=0,
│   │   │   Priority=<EventPriority.DEBUG: 1>,
│   │   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   │   TimestampType=None
│   │   ),
│   │   Status=<Status.Success: 0>,
│   │   Data=TestEvent(
│   │   │   arg1=0,
│   │   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   │   arg3=False,
│   │   │   arg4=SimpleStruct(
│   │   │   │   a=71,
│   │   │   │   b=False,
│   │   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   │   d=b'',
│   │   │   │   e='',
│   │   │   │   f=0,
│   │   │   │   g=4.0178029569121155e-41,
│   │   │   │   h=6.0841893833102e-310
│   │   │   ),
│   │   │   arg5=[],
│   │   │   arg6=[]
│   │   )
│   ),
│   EventReadResult(
│   │   Header=EventHeader(
│   │   │   EndpointId=1,
│   │   │   Event=None,
│   │   │   EventNumber=1,
│   │   │   Priority=<EventPriority.DEBUG: 1>,
│   │   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   │   TimestampType=None
│   │   ),
│   │   Status=<Status.Success: 0>,
│   │   Data=TestEvent(
│   │   │   arg1=0,
│   │   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   │   arg3=False,
│   │   │   arg4=SimpleStruct(
│   │   │   │   a=71,
│   │   │   │   b=False,
│   │   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   │   d=b'',
│   │   │   │   e='',
│   │   │   │   f=0,
│   │   │   │   g=4.0178029569121155e-41,
│   │   │   │   h=6.08418940982057e-310
│   │   │   ),
│   │   │   arg5=[],
│   │   │   arg6=[]
│   │   )
│   )
]

### Subscription Interaction

To subscribe to a Node, the same `ReadAttributes` API is used to trigger a subscription, with a valid `reportInterval` tuple passed in being used as a way to indicate the request to create a subscription.

In [18]:
reportingTimingParams = (0, 2) # MinInterval = 0s, MaxInterval = 2s
subscription = await devCtrl.ReadAttribute(1, [(2, Clusters.OnOff)], True, reportingTimingParams)
subscription

<Subscription (Id=4296644708149507576)>

In [19]:
subscription.GetAttributes()

{
│   2: {
│   │   <class 'chip.clusters.Objects.OnOff'>: OnOff(
│   │   │   onOff=False,
│   │   │   globalSceneControl=True,
│   │   │   onTime=0,
│   │   │   offWaitTime=0,
│   │   │   startUpOnOff=0,
│   │   │   attributeList=None,
│   │   │   featureMap=0,
│   │   │   clusterRevision=4
│   │   )
│   }
}

In [20]:
await devCtrl.SendCommand(1, 2, Clusters.OnOff.Commands.On())

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>,
│   'Value': 0
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>,
│   'Value': True
}

Attribute Changed:


...

{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>,
│   'Value': 0
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>,
│   'Value': True
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>,
│   'Value': True
}

In [21]:
await devCtrl.SendCommand(1, 2, Clusters.OnOff.Commands.Off())

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>,
│   'Value': 0
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>,
│   'Value': False
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>,
│   'Value': 0
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>,
│   'Value': False
}

In [23]:
subscription.Shutdown()

#### Subscribe to Events

In [24]:
subscription = await devCtrl.ReadEvent(1, [('*')], reportingTimingParams)

In [29]:
subscription.GetEvents()

[
│   EventReadResult(
│   │   Header=EventHeader(
│   │   │   EndpointId=1,
│   │   │   Event=None,
│   │   │   EventNumber=0,
│   │   │   Priority=<EventPriority.DEBUG: 1>,
│   │   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   │   TimestampType=None
│   │   ),
│   │   Status=<Status.Success: 0>,
│   │   Data=TestEvent(
│   │   │   arg1=0,
│   │   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   │   arg3=False,
│   │   │   arg4=SimpleStruct(
│   │   │   │   a=71,
│   │   │   │   b=False,
│   │   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   │   d=b'',
│   │   │   │   e='',
│   │   │   │   f=0,
│   │   │   │   g=4.0178029569121155e-41,
│   │   │   │   h=6.0841893833102e-310
│   │   │   ),
│   │   │   arg5=[],
│   │   │   arg6=[]
│   │   )
│   ),
│   EventReadResult(
│   │   Header=EventHeader(
│   │   │   EndpointId=1,
│   │   │   Event=None,
│   │   │   EventNumber=1,
│   │   │   Priority=<EventPriority.DEBUG: 1>,
│   │   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   │   TimestampType=None
│   │   ),
│   │   Status=<Status.Success: 0>,
│   │   Data=TestEvent(
│   │   │   arg1=0,
│   │   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   │   arg3=False,
│   │   │   arg4=SimpleStruct(
│   │   │   │   a=71,
│   │   │   │   b=False,
│   │   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   │   d=b'',
│   │   │   │   e='',
│   │   │   │   f=0,
│   │   │   │   g=4.0178029569121155e-41,
│   │   │   │   h=6.08418940982057e-310
│   │   │   ),
│   │   │   arg5=[],
│   │   │   arg6=[]
│   │   )
│   ),
│   EventReadResult(
│   │   Header=EventHeader(
│   │   │   EndpointId=1,
│   │   │   Event=None,
│   │   │   EventNumber=2,
│   │   │   Priority=<EventPriority.DEBUG: 1>,
│   │   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   │   TimestampType=None
│   │   ),
│   │   Status=<Status.Success: 0>,
│   │   Data=TestEvent(
│   │   │   arg1=0,
│   │   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   │   arg3=False,
│   │   │   arg4=SimpleStruct(
│   │   │   │   a=71,
│   │   │   │   b=False,
│   │   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   │   d=b'',
│   │   │   │   e='',
│   │   │   │   f=0,
│   │   │   │   g=4.0178029569121155e-41,
│   │   │   │   h=6.08418943633095e-310
│   │   │   ),
│   │   │   arg5=[],
│   │   │   arg6=[]
│   │   )
│   ),
│   EventReadResult(
│   │   Header=EventHeader(
│   │   │   EndpointId=1,
│   │   │   Event=None,
│   │   │   EventNumber=3,
│   │   │   Priority=<EventPriority.DEBUG: 1>,
│   │   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   │   TimestampType=None
│   │   ),
│   │   Status=<Status.Success: 0>,
│   │   Data=TestEvent(
│   │   │   arg1=0,
│   │   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   │   arg3=False,
│   │   │   arg4=SimpleStruct(
│   │   │   │   a=71,
│   │   │   │   b=False,
│   │   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   │   d=b'',
│   │   │   │   e='',
│   │   │   │   f=0,
│   │   │   │   g=4.0178029569121155e-41,
│   │   │   │   h=6.08418943633095e-310
│   │   │   ),
│   │   │   arg5=[],
│   │   │   arg6=[]
│   │   )
│   )
]

In [34]:
# Force an event to get emitted.
await devCtrl.SendCommand(1, 1, Clusters.TestCluster.Commands.TestEmitTestEventRequest())

TestEmitTestEventResponse(
│   value=6
)

Now, data comes...

In [35]:
subscription

<Subscription (Id=15289740281926510516)>

Received Event:


EventReadResult(
│   Header=EventHeader(
│   │   EndpointId=1,
│   │   Event=None,
│   │   EventNumber=6,
│   │   Priority=<EventPriority.DEBUG: 1>,
│   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   TimestampType=None
│   ),
│   Status=<Status.Success: 0>,
│   Data=TestEvent(
│   │   arg1=0,
│   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   arg3=False,
│   │   arg4=SimpleStruct(
│   │   │   a=71,
│   │   │   b=False,
│   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   d=b'',
│   │   │   e='',
│   │   │   f=0,
│   │   │   g=4.0178029569121155e-41,
│   │   │   h=6.08418943633095e-310
│   │   ),
│   │   arg5=[],
│   │   arg6=[]
│   )
)